In [1]:
import ast
import os
import time
import json
from openai import OpenAI
from tqdm import tqdm

In [2]:
#settings
debug=False
assistant_id = os.environ.get("ASSISTANT_ID")
sleep_time_for_openai_to_respond = 5
validateResponse = 'request_response.json' #Name of the json file

In [3]:
#instantiate

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

assistant = client.beta.assistants.retrieve(assistant_id)
if debug:
    print(f'Assistant id is : {assistant.id}')
    
thread = client.beta.threads.create()
if debug:
    print(f'Thread id is : {thread.id}')


f = open(validateResponse)
requests = json.load(f)
if debug:
    print(requests)


In [4]:
if debug:
    for qn in requests:
        print(qn['question'])

In [5]:
for qn in tqdm(requests):
    thread_message = client.beta.threads.messages.create(
      thread.id,
      role="user",
      content=qn['question']
    )
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )
    if debug:
        print(f'Run id : {run.id}')
    time.sleep(sleep_time_for_openai_to_respond)
    run = client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )

    if debug:
        print(f'Finished question: {qn["qn_id"]}')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:51<00:00,  5.74s/it]


In [6]:
thread_messages = client.beta.threads.messages.list(thread.id)
if debug:
    print('Messages ')
    print(thread_messages.data)

In [7]:
counter = 0
for message in tqdm(reversed(thread_messages.data)):
    
    
    if message.role == 'assistant':
        qn = requests[counter]
        keywords = qn['Keywords']
        for keyword in keywords:
            #print(keyword)
            if keyword not in message.content[0].text.value:
                
                print(f'For question number "{counter+1}": {qn["question"]}\n Keyword {keyword} is not found')
                print("Response from assistant: \n" + message.content[0].text.value)
        counter = counter+1

18it [00:00, 123969.58it/s]
